In [1]:
#paquetes python
import os
import numpy as np
import copy
import pprint
from math import sqrt
from scipy.linalg import solve_triangular
import inspect
import sys

#paquetes funciones definidas obtenidas de archivos .py
import funciones_factorizacion_QR as fQR

**OBJETIVO:**

El presente notebook tiene por finalidad exhibir y poner a prueba funciones definidas por el equipo QR para resolver un Sistema de Ecuaciones Lineales (SEL), a través de 3 pasos:

    1) Obtención de factorización QR (A=QR)
    2) Implementación del método de eliminación por bloques
    3) Resolución de SEL (Ax=b) con los 2 pasos anteriores
    
Para lograr tal objetivo se presentarán a continuacion las funciones implementadas en python, así como una serie de ejemplos para entender mejor su funcionamiento

## Factorización QR

### Funciones

In [2]:
lines=inspect.getsource(fQR.crear_matriz_aleatoria)
print(lines)

def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias

    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if isinstance(renglones, int)==False or isinstance(columnas, int)==False:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')
    elif renglones<=0 or columnas<=0:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')            
    if isinstance(maximo_valor, (int, float))==False or isinstanc

In [3]:
help(fQR.crear_matriz_aleatoria)

Help on function crear_matriz_aleatoria in module funciones_factorizacion_QR:

crear_matriz_aleatoria(renglones, columnas, maximo_valor, minimo_valor, entero=False)
    
    Descripción:
            Función que genera matrices de tamaño definido con entradas dentro del rango
            indicado
    
    Parametros:
            renglones       integer indica renglones de la matriz
            columnas        integer indica columnas de la matriz
            maximo_valor    integer o double señala valor máximo de las entradas de la matriz
            minimo_valor    integer o double señala valor mínimo de las entradas de la matriz
            entero          Boolean, indica si las entradas serán enteras (True) o no (False)
    
    Resultado:
            M               np.ndarray que representa matriz (mxn) con numeros al azar en el
                            rango indicado
    
    Ejemplos:
    Crear matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6:
    cr

In [4]:
lines=inspect.getsource(fQR.house)
print(lines)

def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(x) is not np.ndarray:
        sys.exit('x debe ser de tipo numpy.ndarray')
    
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
     

In [5]:
help(fQR.house)

Help on function house in module funciones_factorizacion_QR:

house(x)
    
    Descripción:
            Función que calcula vector de householder
    
    Parametros:
            x       np.ndarray que representa vector al que se le hará la reflexión householder
    
    Resultado:
            Beta    double para obtener matriz de reflexión householder v
            v       np.ndarray que representa vector que representa la reflexión de householder
    
    Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de
    householder R, donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz  
    de pxp si v tiene p entradas). El vector v está construido de tal forma que al generar la 
    matriz de reflexión de householder Rf y multiplicarse por x se obtiene una proyección de 
    este último sobre el vector canónico e_1=(1,0,0,...,0) obteniendo un vector con todas sus 
    entradas ingualas  a cero excepto la primera, que de hecho es ||x||

In [6]:
lines=inspect.getsource(fQR.matriz_auxiliar_Arv)
print(lines)

def matriz_auxiliar_Arv(A):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    
    #m contiene el numero de renglones y n el de columnas
    m=A.shape[0]
    n=A.shape[1]
    #se crea una matriz con los valores de A
    Arv=copy.copy(A)
    for j in range(n):
        beta

In [7]:
help(fQR.matriz_auxiliar_Arv)

Help on function matriz_auxiliar_Arv in module funciones_factorizacion_QR:

matriz_auxiliar_Arv(A)
    
    Descripción:
            Función que genera una matriz que contiene los elementos r distintos de cero de 
            la matriz R y las entradas de los vectores householder v (excepto la primera), 
            con los cuales se puede calcular la matriz Q. Ambas matrices componentes de la 
            factorización QR
    
    Parametros:
            A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener 
                   elemento de factorización QR
    
    Resultado:
            Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes 
                   distintas de cero de la matriz R y los vectores householder con los 
                   que se puede obtener la matriz Q, y con ello la factorización QR
    
    La factorización QR consiste en encontrar 2 matrices tales que A=Q*R 
    donde Q es una matriz ortogonal y R es una matriz 

In [8]:
lines=inspect.getsource(fQR.matriz_R)
print(lines)

def matriz_R(Arv):
    """
    Función que devuelve la matriz R de la factorización QR de una matriz A, 
    apartir de la matriz Arv
    
    params: Arv   Matriz (mxn) que incluye elementos de la matriz R y Q, de la factorización QR

    return: R     Matriz (mxn) R de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')

    m=Arv.shape[0]
    n=Arv.shape[1]
    R=np.zeros((m,n))
    #la matriz Arv ya tiene los elementos de r en el triangulo superior
    #por lo que únicamente se sustraen dichos valores y lo demás se deja en ceros
    for j in range(n):
        R[0:(j+1),j]=Arv[0:(j+1),j]
    return R



In [9]:
help(fQR.matriz_R)

Help on function matriz_R in module funciones_factorizacion_QR:

matriz_R(Arv)
    
    Descripción:
            Función que devuelve la matriz R de la factorización QR de una matriz A,
            apartir de la matriz Arv
    
    Parametros:
            Arv   np.ndarray que representa matriz (mxn) que incluye elementos de 
                  la matriz R y Q, de la factorización QR
    
    Resultado:
            R     np.ndarray que representa matriz (mxn) R de la factorización A=QR
    
    Visualmente la sustracción de los elementos de R se ve así:
         [ r11  r12  r13  r14 ]            [ r11  r12  r13  r14 ]
         [ v21  r22  r23  r24 ]            [  0   r22  r23  r24 ]
    Arv= [ v31  v32  r33  r34 ]   -->   R= [  0    0   r33  r34 ]
         [ v41  v42  v43  r44 ]            [  0    0    0   r44 ]
         [ v51  v52  v53  v54 ]            [  0    0    0    0  ]



In [10]:
lines=inspect.getsource(fQR.matriz_Q)
print(lines)

def matriz_Q(Arv):
    """
    Función que devuelve la matriz R de la factorización QR de una matriz A,
    apartir de la matriz Arv
    
    params: Arv  Matriz (mxn) con la info escencial para la factorización

    return: Q    Matriz Q (mxm) de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
        
    m=Arv.shape[0]
    n=Arv.shape[1]
    Q=np.eye(m)
    I=np.eye(m)
    for j in range(n-1,-1,-1):
        #Se sustrae la información de los vectores de householder contenida
        #en la matriz Arv, agregando la primera entrada que no está en dicha matriz (y que es 1)
        v=np.concatenate((1,Arv[(j+1):m,j]), axis=None)
        #Se calcula el factor beta para obtener la matriz de reflexión
        beta

In [11]:
help(fQR.matriz_Q)

Help on function matriz_Q in module funciones_factorizacion_QR:

matriz_Q(Arv)
    
    Descripción:
            Función que devuelve la matriz R de la factorización QR de una matriz A,
            apartir de la matriz Arv
    
    Parametros:
            Arv   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                  elementos de la matriz R y Q, de la factorización QR
    
    Resultado:
            R     np.ndarray que representa matriz Q (mxm) de la factorización A=QR
    
    La matriz Q (matriz ortogonal tal que Q_transpuesta*Q es igual a la identidad (mxm)
    se calcula como el producto de las matrices Q_j's, las cuales se obtienen a partir
    de las diferentes matrices de reflexión householder utilizadas para generar los ceros
    en las columnas de A para formar la matriz R
    
    Q = Q_1*Q_2*Q_3*Q_4
    
    Nota: Cada matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo 
    vector de householder como: Q_j = Identidad(mxm) - Beta*v_

In [12]:
lines=inspect.getsource(fQR.Q_j)
print(lines)

def Q_j(Arv,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: Arv   Matriz (mxn) con la info escencial
            j     indica el índice de la matriz Q_j

    return: Qj    Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
    if isinstance(j, int)==False:
        sys.exit('El parámetro j deber ser un entero')
    elif 1>j or j>Arv.shape[1]:
        sys.exit('El parámetro j debe estar en el rango [1,no. columnas de Arv]')
    
    m=Arv.shape[0]
    n=Arv.shape[1]
    Qj=np.eye(m)
 

In [13]:
help(fQR.Q_j)

Help on function Q_j in module funciones_factorizacion_QR:

Q_j(Arv, j)
    
    Descripción:
            Función que calcula la matriz Qj (en el proceso de obtención de 
            factorización QR se van obteniendo n Qj's, que si se multiplican todas 
            da por resultado Q=Q_1*Q_2*...*Q_n)
    
    Parámetros:                            
            A_r_v   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                elementos de la matriz R y Q, de la factorización QR
            j       integer que indica el índice de la matriz Q_j
    
    Resultado:
            Qj      np.ndarray que representa la j-esima transformación de householder
    
    La matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo vector 
    de householder como: Q_j = Identidad(mxm) - Beta*v_j*v_j_transpuesta



## Factorización QR

### Ejemplos

In [14]:
#Pra probar el funcionamiento de estas funciones puede definirse una matriz entrada por entrada
#o usar la función crear_matriz_aleatoria

#Ejemplo a mano (asegurandose de que sus entradas sean tipo dobles -aunque sólo se capture
#una parte entera-, pues de lo contrario se tendrán errores por redondeo significativos)
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [15]:
x=np.array([1,2,3.])
print("x=\n",x)

Beta,v=fQR.house(x)
print("\n\nBeta=\n",np.round(Beta,4))
print("\n\nv=\n",np.round(v,4))

#Calculando matriz de reflexión de householder R
R=np.eye(3)-Beta*np.outer(v,v)
print("\n\nR=\n",np.round(R,4))

#Observese como el producto de Rx da por resultado un vector con
#todas sus entradas igualas a cero excepto la primera
print("\n\nRx=\n",np.round(R@x,4))

x=
 [1. 2. 3.]


Beta=
 0.7327


v=
 [ 1.     -0.7295 -1.0942]


R=
 [[ 0.2673  0.5345  0.8018]
 [ 0.5345  0.6101 -0.5849]
 [ 0.8018 -0.5849  0.1227]]


Rx=
 [3.7417 0.     0.    ]


In [16]:
#Definase una matriz A
A=fQR.crear_matriz_aleatoria(7,4,6,-6)
print('\n\nA=\n',np.round(A,4))

#Calculamos la matriz auxiliar Arv
Arv=fQR.matriz_auxiliar_Arv(A)
print('\n\nArv=\n',np.round(Arv,4))

#Obtenemos la matriz Q y R a partir de Arv
Q=fQR.matriz_Q(Arv)
print('\n\nQ=\n',np.round(Q,4))
R=fQR.matriz_R(Arv)
print('\n\nR=\n',np.round(R,4))

#Checamos que QR=A
print('\n\nQ@R=\n',np.round(Q@R,4))



A=
 [[ 5.399   1.037  -5.6713  5.3381]
 [-3.2108  0.5237 -3.9984 -0.4543]
 [-3.7008 -2.0237 -3.7576  1.0023]
 [-5.328   5.8827 -3.7054 -3.9283]
 [-2.9431  5.8806  3.3777 -0.5544]
 [-1.442   2.3905 -5.234  -5.0265]
 [-2.156  -2.6865 -0.0525  4.959 ]]


Arv=
 [[ 9.8454 -3.5447  1.3801  4.6405]
 [ 0.7221  8.6602 -1.1298 -3.6938]
 [ 0.8323  0.51   10.5653 -0.3852]
 [ 1.1983 -0.0343 -0.5028  7.7444]
 [ 0.6619 -0.2488 -0.8968  0.0679]
 [ 0.3243 -0.079   0.289   0.3829]
 [ 0.4849  0.4288 -0.2517 -0.2596]]


Q=
 [[ 0.5484  0.3442 -0.5716  0.4964  0.07   -0.0523 -0.0014]
 [-0.3261 -0.073  -0.3437  0.0848 -0.2833  0.3024 -0.769 ]
 [-0.3759 -0.3875 -0.348   0.1525  0.6626 -0.3534 -0.0145]
 [-0.5412  0.4578 -0.2311  0.0239 -0.4055 -0.4597  0.2606]
 [-0.2989  0.5567  0.4183  0.3939  0.4272  0.2716 -0.1201]
 [-0.1465  0.2161 -0.4532 -0.4808  0.2303  0.5759  0.3327]
 [-0.219  -0.3998 -0.0191  0.5799 -0.2753  0.4053  0.4642]]


R=
 [[ 9.8454 -3.5447  1.3801  4.6405]
 [ 0.      8.6602 -1.1298 -3.6938

In [17]:
#Comparemos con el código de numpy
Q_np, R_np=np.linalg.qr(A)
#notamos que numpy obtiene con signos contrarios Q y R
print('Q_np=\n',np.round(Q_np,4))
#Además R no guarda toda la matriz R de mxn, sino sólo la parte superior de nxn
print('\nR_np=\n',np.round(R_np,4))
#la multiplicación da por resultado A
print('\nQ_np@R_np=\n',np.round(Q_np@R_np,4))

Q_np=
 [[-0.5484  0.3442 -0.5716 -0.4964]
 [ 0.3261 -0.073  -0.3437 -0.0848]
 [ 0.3759 -0.3875 -0.348  -0.1525]
 [ 0.5412  0.4578 -0.2311 -0.0239]
 [ 0.2989  0.5567  0.4183 -0.3939]
 [ 0.1465  0.2161 -0.4532  0.4808]
 [ 0.219  -0.3998 -0.0191 -0.5799]]

R_np=
 [[-9.8454  3.5447 -1.3801 -4.6405]
 [ 0.      8.6602 -1.1298 -3.6938]
 [ 0.      0.     10.5653 -0.3852]
 [ 0.      0.      0.     -7.7444]]

Q_np@R_np=
 [[ 5.399   1.037  -5.6713  5.3381]
 [-3.2108  0.5237 -3.9984 -0.4543]
 [-3.7008 -2.0237 -3.7576  1.0023]
 [-5.328   5.8827 -3.7054 -3.9283]
 [-2.9431  5.8806  3.3777 -0.5544]
 [-1.442   2.3905 -5.234  -5.0265]
 [-2.156  -2.6865 -0.0525  4.959 ]]


In [18]:
#Observese que Q=(Q_4*Q_3*Q_2*Q_1)_transpuesta
np.round(np.transpose(fQR.Q_j(Arv,4)@fQR.Q_j(Arv,3)@fQR.Q_j(Arv,2)@fQR.Q_j(Arv,1)),4)

array([[ 0.5484,  0.3442, -0.5716,  0.4964,  0.07  , -0.0523, -0.0014],
       [-0.3261, -0.073 , -0.3437,  0.0848, -0.2833,  0.3024, -0.769 ],
       [-0.3759, -0.3875, -0.348 ,  0.1525,  0.6626, -0.3534, -0.0145],
       [-0.5412,  0.4578, -0.2311,  0.0239, -0.4055, -0.4597,  0.2606],
       [-0.2989,  0.5567,  0.4183,  0.3939,  0.4272,  0.2716, -0.1201],
       [-0.1465,  0.2161, -0.4532, -0.4808,  0.2303,  0.5759,  0.3327],
       [-0.219 , -0.3998, -0.0191,  0.5799, -0.2753,  0.4053,  0.4642]])

In [19]:
#Puede comprobarse que Q_transpuesta*Q es igual a la identidad
print(np.round(np.transpose(Q)@Q,4))

[[ 1.  0. -0. -0. -0. -0.  0.]
 [ 0.  1. -0.  0. -0.  0. -0.]
 [-0. -0.  1.  0.  0.  0. -0.]
 [-0.  0.  0.  1.  0. -0. -0.]
 [-0. -0.  0.  0.  1.  0. -0.]
 [-0.  0.  0. -0.  0.  1.  0.]
 [ 0. -0. -0. -0. -0.  0.  1.]]


In [20]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
A=fQR.crear_matriz_aleatoria(5,4,5,-5,True)
print('\n\nA=\n',np.round(A,4))

m=A.shape[0]
n=A.shape[1]
Q=np.eye(m)
R=copy.copy(A)
Arv=fQR.matriz_auxiliar_Arv(A)
for j in range(n):
    if j==0:
        cadena='Q'+str(j+1)
    else:
        cadena='Q'+str(j+1) + ' @ ' + cadena
    Qj=fQR.Q_j(Arv,j+1)
    print('\n',cadena,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=fQR.Q_j(Arv,j+1)@R
    print('\n',cadena,'@ A=\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))



A=
 [[-5. -2.  2. -3.]
 [ 2. -2. -1. -3.]
 [-4.  5.  3.  5.]
 [-1. -4.  1.  5.]
 [-5. -4.  5.  4.]]

 Q1 :
 [[-0.5934  0.2374 -0.4747 -0.1187 -0.5934]
 [ 0.2374  0.9646  0.0707  0.0177  0.0884]
 [-0.4747  0.0707  0.8586 -0.0354 -0.1768]
 [-0.1187  0.0177 -0.0354  0.9912 -0.0442]
 [-0.5934  0.0884 -0.1768 -0.0442  0.779 ]]

 Q1 @ A=
 [[ 8.4261  1.1868 -5.9339 -4.2724]
 [-0.     -2.4747  0.1819 -2.8105]
 [ 0.      5.9494  0.6363  4.6209]
 [ 0.     -3.7626  0.4091  4.9052]
 [ 0.     -2.8132  2.0454  3.5261]]

 Q2 @ Q1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.     -0.3103  0.7461 -0.4718 -0.3528]
 [ 0.      0.7461  0.5752  0.2687  0.2009]
 [ 0.     -0.4718  0.2687  0.8301 -0.127 ]
 [ 0.     -0.3528  0.2009 -0.127   0.905 ]]

 Q2 @ Q1 @ A=
 [[ 8.4261  1.1868 -5.9339 -4.2724]
 [ 0.      7.9744 -0.4963  0.7612]
 [ 0.     -0.      1.0224  2.5873]
 [ 0.      0.      0.1649  6.1914]
 [ 0.      0.      1.8628  4.4877]]

 Q3 @ Q2 @ Q1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.

In [21]:
#Se prueba un ejemplo más grande la función matriz_auxiliar_Arv
#que es base para la obtención de Q y R
A=fQR.crear_matriz_aleatoria(11**2,10**2,10,-10,False)
Arv=fQR.matriz_auxiliar_Arv(A)
Arv

array([[ 6.52486441e+01,  8.80122622e+00, -5.00032819e+00, ...,
        -8.20571879e+00, -7.95791242e+00,  1.02650296e+01],
       [-7.67409779e-02,  6.61415075e+01,  4.44068162e+00, ...,
        -3.41337515e-01,  4.86694878e+00, -1.02706975e+01],
       [ 9.22344768e-02, -1.08416676e-01,  6.22927306e+01, ...,
        -5.86180586e+00, -1.84837765e+00,  5.04938324e+00],
       ...,
       [-4.18395394e-02,  1.15686368e-01,  2.55152744e-02, ...,
        -3.58366408e-01,  2.78803608e-01,  2.55153635e-01],
       [-1.41638976e-02, -2.94026568e-02, -1.03628232e-01, ...,
        -3.81759477e-02,  2.27571558e-02,  2.01055870e-02],
       [ 1.15152902e-01,  1.11095444e-01,  6.86910962e-02, ...,
         7.85969031e-02, -2.89768803e-01,  7.66046813e-02]])

## Solución de Sistemas de Ecuaciones Lineales

### Funciones

In [22]:
lines=inspect.getsource(fQR.Solucion_SEL_QR_nxn)
print(lines)

def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    Arv=matriz_auxiliar_Arv(A)
    m=Arv.shape[0]
    n=Arv.shape[0]
    Q=matriz_Q(Arv)
    R=matriz_R(Arv)
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x



In [23]:
help(fQR.Solucion_SEL_QR_nxn)

Help on function Solucion_SEL_QR_nxn in module funciones_factorizacion_QR:

Solucion_SEL_QR_nxn(A, b)
    
    Descripción:
            Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con 
        n ecuaciones y n incognitas
    
    Parámetros:                            
            A      Matriz (nxn) que representa los coeficientas de las ecuaciones 
               elementos de la matriz R y Q, de la factorización QR
            b      vector (nx1) constantes del sistema
    
    Resultado:
            x      vector que satisface (Ax=b)
    
    El Sistema de Ecuaciones Lineales establecido como Ax=b, es resuelto con esta función
    mediante la factorización QR como a continuación se explica:
    
      Si A*x = b y A = Q*R, con A matriz (mxn), b (mx1) y Q una matriz ortogonal (mxm) y 
                                R (mxn) una matriz triangular superior
      entonces Q*R*x = b.
      
      Aprovechando la propiedad de Q^(-1)=Q, es decir, Q^(-1)*Q=Ide

## Eliminación por bloques

### Funciones

In [24]:
lines=inspect.getsource(fQR.crear_bloques)
print(lines)

def crear_bloques(A, b=True, m1=True, n1=True):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    m1 = m//2 if m1 is True else m1
    print(m1)
    n1 = n//2 if n1 is True else n1
    print(n1)

    # Los bloques deben cumplir la condicion de tamaño
    if m1 >

In [25]:
help(fQR.crear_bloques)

Help on function crear_bloques in module funciones_factorizacion_QR:

crear_bloques(A, b=True, m1=True, n1=True)
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido



In [26]:
lines=inspect.getsource(fQR.eliminacion_bloques)
print(lines)

def eliminacion_bloques(A,b, m1=True, n1=True):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A21,A12,A22,b1,b2 = crear_bloques(A,b,m1,n1)

    if np.linalg.det(A11)==0:
        sys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y 

In [27]:
help(fQR.eliminacion_bloques)

Help on function eliminacion_bloques in module funciones_factorizacion_QR:

eliminacion_bloques(A, b, m1=True, n1=True)
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques



## Solución de Sistemas de Ecuaciones Lineales

### Ejemplos

### A) Sistema con soluciones únicas

In [28]:
#Generamos un sistema de ecuaciones lineales
m=6
n=6
A=np.round(fQR.crear_matriz_aleatoria(m,n,6,-6,False),2)
b = np.round(fQR.crear_matriz_aleatoria(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[-0.91  3.04  4.13  1.86 -0.58  1.34]
 [ 4.81  4.7  -5.4   3.45 -1.61  3.07]
 [-3.78 -0.91 -4.15 -2.8   1.86 -5.16]
 [ 1.34  5.59  1.14  1.32  1.2   3.5 ]
 [ 1.24  1.12  4.88 -0.02  3.34  3.48]
 [-1.56 -1.11  5.59  1.57  1.87  2.44]]
b: [[ 1.01]
 [-5.88]
 [ 4.48]
 [ 0.35]
 [-5.17]
 [-2.54]]


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [29]:
np.linalg.solve(A,b)

array([[-2.62198785],
       [ 0.56546405],
       [-0.57720686],
       [-2.47897408],
       [-2.04539603],
       [ 2.02493044]])

Ahora usemos la factorización QR

In [30]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

array([[ 0.25431183],
       [ 0.92214874],
       [ 0.78094419],
       [-0.29174144],
       [-0.52404568],
       [-2.02493044]])

In [31]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

array([[ 2.84510853],
       [-5.03909052],
       [ 5.24941968],
       [-1.71534438],
       [-3.63207478],
       [-3.4336633 ]])

Por último usemos eliminación por bloques con QR

In [32]:
fQR.eliminacion_bloques(A,b)

3
3


array([[-2.35223007],
       [ 0.97231599],
       [ 0.59337237],
       [-3.77497903],
       [-4.03990252],
       [ 2.0631865 ]])

### B) Sistema sin solución

In [33]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [5]], dtype='d')
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

A:
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])
b:
array([[3.],
       [3.],
       [5.]])


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [34]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

* Marca un error: "Singular matrix"

Ahora usemos la factorización QR

In [ ]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

In [ ]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

Por último usemos eliminación por bloques con QR

In [ ]:
fQR.eliminacion_bloques(A,b)

* Se concluye que es un sistema de rectas paralelas, por lo tanto no existe solúción.

### C) Sistema con infinitas soluciones

In [ ]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [0]], dtype='d')
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [ ]:
np.linalg.solve(A,b)

* Marca un error: "Singular matrix"

Ahora usemos la factorización QR

In [ ]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

In [ ]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

Por último usemos eliminación por bloques con QR

In [ ]:
fQR.eliminacion_bloques(A,b)